In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install "git+https://github.com/albumentations-team/albumentations.git"

In [ ]:
!pip install timm

In [1]:
import sys
sys.path.append("/content/drive/MyDrive/Kaggle/Sartorius")
sys.path.append("/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/swin_L")

In [2]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer
from albumentations import *
import torch
import os
from detectron2.data import detection_utils
from utils.aug import MyMapper
from utils.add_swint_config import add_swint_config
from detectron2.solver.build import *
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import warnings
import swint
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
class Config:
  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]
  anchor_generators_sizes = [[8], [16], [32], [64],[128]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]
  model_config = "/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/swin_L/detectron2/configs/SwinT/mask_rcnn_swint_T_FPN_3x.yaml"
  model_weights = "/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/swin_L/swin_L_384_22k.pth"

In [4]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
 
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    @classmethod
    def build_optimizer(cls, cfg, model):
        params = get_default_optimizer_params(
            model,
            base_lr=cfg.SOLVER.BASE_LR,
            weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
            bias_lr_factor=cfg.SOLVER.BIAS_LR_FACTOR,
            weight_decay_bias=cfg.SOLVER.WEIGHT_DECAY_BIAS,
        )

        def maybe_add_full_model_gradient_clipping(optim):  # optim: the optimizer class
            # detectron2 doesn't have full model gradient clipping now
            clip_norm_val = cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE
            enable = (
                cfg.SOLVER.CLIP_GRADIENTS.ENABLED
                and cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE == "full_model"
                and clip_norm_val > 0.0
            )

            class FullModelGradientClippingOptimizer(optim):
                def step(self, closure=None):
                    all_params = itertools.chain(*[x["params"] for x in self.param_groups])
                    torch.nn.utils.clip_grad_norm_(all_params, clip_norm_val)
                    super().step(closure=closure)

            return FullModelGradientClippingOptimizer if enable else optim

        optimizer_type = cfg.SOLVER.OPTIMIZER
        if optimizer_type == "SGD":
            optimizer = maybe_add_gradient_clipping(torch.optim.SGD)(
                params, cfg.SOLVER.BASE_LR, momentum=cfg.SOLVER.MOMENTUM,
                nesterov=cfg.SOLVER.NESTEROV,
                weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            )
        elif optimizer_type == "AdamW":
              optimizer = maybe_add_full_model_gradient_clipping(torch.optim.AdamW)(
                params, cfg.SOLVER.BASE_LR, betas=(0.9, 0.999),
                weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            )
        else:
            raise NotImplementedError(f"no optimizer type {optimizer_type}")
        return optimizer

     # @classmethod
    # def build_train_loader(cls, cfg, sampler=None):
    #     return build_detection_train_loader(
    #         cfg, mapper=MyMapper(cfg), sampler=sampler
    #     )

    # def build_hooks(self):
    #   # copy of cfg
    #   cfg = self.cfg.clone()

    #   # build the original model hooks
    #   hooks = super().build_hooks()

    #   # add the best checkpointer hook
    #   hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
    #                                     DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
    #                                     "MaP IoU",
    #                                     "max",
    #                                     ))
      
def setup():
  cfg = get_cfg()
  add_swint_config(cfg)
  cfg.merge_from_file(Config.model_config)
  #cfg.MODEL.WEIGHTS = Config.model_weights
  #cfg.merge_from_list(args.opts)
  #cfg.freeze()
  #default_setup(cfg, args)
  return cfg


In [8]:
def run():
  dataDir = "/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation"
  DatasetCatalog.clear()
  MetadataCatalog.clear()
  register_coco_instances(f'sartorius_train',{}, '/content/drive/MyDrive/Kaggle/Sartorius/input/fixed_fold/coco_cell_train_fold{}.json'.format(fold), dataDir)
  register_coco_instances(f'sartorius_val',{},'/content/drive/MyDrive/Kaggle/Sartorius/input/fixed_fold/coco_cell_valid_fold{}.json'.format(fold), dataDir)

  #cfg = get_cfg()
  #cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"))
  cfg = setup()
  cfg.INPUT.MIN_SIZE_TRAIN = (220, 240, 260)
  cfg.SOLVER.BASE_LR = 5e-4
  cfg.DATASETS.TRAIN = (f"sartorius_train",)
  cfg.DATASETS.TEST = (f"sartorius_val",)
  #cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
  cfg.SOLVER.MAX_ITER = len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH * 20 #尝试20ep 用lr调整
  #cfg.SOLVER.STEPS = (cfg.SOLVER.MAX_ITER//3,cfg.SOLVER.MAX_ITER*2//3)
  cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  #cfg.MODEL.BACKBONE.FREEZE_AT = 2
  #cfg.MODEL.RESNETS.DEPTH = 101
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
  # cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS = (0.5, 0.6, 0.7)
  

  cfg.OUTPUT_DIR = "/content/drive/MyDrive/Kaggle/Sartorius/model/swinT/fold_{}#".format(fold)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [9]:
# 2 的理论score最高
for fold in range(1,6):
  run()

[12/17 05:19:38 d2.data.datasets.coco]: Loading /content/drive/MyDrive/Kaggle/Sartorius/input/fixed_fold/coco_cell_train_fold1.json takes 10.71 seconds.
[12/17 05:19:38 d2.data.datasets.coco]: Loaded 484 images in COCO format from /content/drive/MyDrive/Kaggle/Sartorius/input/fixed_fold/coco_cell_train_fold1.json
[12/17 05:19:39 d2.data.datasets.coco]: Loaded 484 images in COCO format from /content/drive/MyDrive/Kaggle/Sartorius/input/fixed_fold/coco_cell_train_fold1.json
[12/17 05:19:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3

Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.norm0.{bias, weight}
backbone.bottom_up.norm1.{bias, weight}
backbone.bottom_up.norm2.{bias, weight}
backbone.bottom_up.norm3.{bias, weight}
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.

[12/17 05:19:42 d2.engine.train_loop]: Starting training from iteration 0
ERROR [12/17 05:19:55 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/detectron2/engine/train_loop.py", line 149, in train
    self.run_step()
  File "/usr/local/lib/python3.7/dist-packages/detectron2/engine/defaults.py", line 494, in run_step
    self._trainer.run_step()
  File "/usr/local/lib/python3.7/dist-packages/detectron2/engine/train_loop.py", line 395, in run_step
    loss_dict = self.model(data)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/detectron2/modeling/meta_arch/rcnn.py", line 163, in forward
    _, detector_losses = self.roi_heads(images, features, proposals, gt_instances)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_imp

RuntimeError: ignored

In [ ]:
!git clone https://github.com/xiaohu2015/SwinT_detectron2.git